# JupyterLabを松尾研プレ環境でセットアップしてAsk-LLMを実行

## 重要なドキュメント

以下2つの松尾研提供のドキュメントを必ず読む。もし今読んでいるドキュメントとの矛盾があれば、以下の2つのドキュメントの記述を優先する(特にサーバ利用ルールが流動的に変わるので常に最新の情報を確認)。

- 松尾研GCPサーバ利用手順書（プレ環境）
  - https://docs.google.com/document/d/1kJiRqUFSm6pMFoIX4hqx1CxAIa5MBSwC9DndzB6pxqE/edit?usp=sharing
- 松尾研リポジトリのインフラ構築手順
  - https://github.com/matsuolab/ucllm_nedo_prod/blob/main/infra/README.md

## 初回のセットアップ手順

### gcloud CLI のインストール

ローカルマシンに gcloud CLI をインストール。

- https://cloud.google.com/sdk/docs/install

### sshでログインノードに接続

ローカルマシンのターミナルから以下のコマンドを実行してログインノードに接続。

```bash
export ZONE="asia-southeast1-a"
export INSTANCE="mlpre-login-..."            # ログインノードのインスタンス名
export PROJECT="g..."                        # GCPプロジェクト名
```

```bash
gcloud compute ssh --zone $ZONE $INSTANCE --project $PROJECT
```

### 計算ノードにbashでログイン

計算ノードにbashでログインする。

```bash
srun --partition g2 --nodes=1 --gpus-per-node=1 --time=03:00:00 --pty bash -i
```

`squeue` して計算ノードの名前 `mlpre-g2-ghpc-...` を確認。計算ノードのマシン名 `mlpre-g2-...` は後でsshポートフォワーディングで使うのでメモしておく。

### 共有ディスクに実験用ディレクトリを作成

共有ディスクに実験用ディレクトリを作成。ディレクトリの場所やパーミッションはチームのルールに従う。

```bash
mkdir -p /persistentshare/storage/team_.../...
chmod 700 /persistentshare/storage/team_.../...
```

環境変数で実験用ディレクトリの場所を指定。以下を実行するとともに `~/.bashrc` の末尾に追加。

```bash
export EXP_HOME="/persistentshare/storage/team_.../..."
```

### Python 実行環境の構築

松尾研マニュアルの以下ページを参考に EXP_HOME 以下に Miniconda をインストール。

- https://github.com/matsuolab/ucllm_nedo_prod/blob/main/infra/README.md#environment-preparation

```bash
mkdir -p $EXP_HOME/miniconda3
cd $EXP_HOME/miniconda3

wget https://repo.anaconda.com/miniconda/Miniconda3-py310_23.10.0-1-Linux-x86_64.sh
bash Miniconda3-py310_23.10.0-1-Linux-x86_64.sh -h  # オプションを確認
# -b オプションでライセンスに同意したことになることに注意
bash Miniconda3-py310_23.10.0-1-Linux-x86_64.sh -b -u -p $EXP_HOME/miniconda3

source $EXP_HOME/miniconda3/etc/profile.d/conda.sh

which conda      # $EXP_HOME/miniconda3/condabin/conda
conda --version  # conda 23.10.0
```

conda環境の作成。

```bash
conda create --name jupyter39 python=3.9
# conda create --name jupyter310 python=3.10
```

conda環境をアクティベートする。

```bash
conda activate jupyter39
# conda activate jupyter310
```

CUDAとPyTorchのインストール。

```bash
conda install nvidia/label/cuda-11.8.0::cuda-toolkit -y

conda install pytorch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 pytorch-cuda=11.8 -c pytorch -c nvidia -y
```

JupyterLabのインストール。

```bash
conda install -c conda-forge jupyterlab -y
which jupyter-lab  # $HOME/miniconda3/envs/py310/bin/jupyter-lab
jupyter-lab --version  # 4.1.5
```

### JupyterLabの動作確認

JupyterLabを手動で起動して動作確認する。

```bash
jupyter-lab --no-browser --port 8888 --ip $(hostname -i)
```

ログに `http://127.0.0.1:8888/lab?token=...` のような行が出るので、クリップボードにコピーしておく。

**今作業しているターミナルとは別のローカルマシンのターミナル**から、以下のコマンドを実行してローカルマシンのポート8888を計算ノード`mlpre-g2-...`のポート8888にフォワードする。`--` 以降のオプションは通常のsshコマンドのオプション。計算ノード名は `squeue` で確認する。

```bash
export ZONE="asia-southeast1-a"
export INSTANCE="mlpre-login-..."       # ログインノードのインスタンス名
export PROJECT="g..."                   # プロジェクト名
export COMPUTE_INSTANCE="mlpre-g2-..."  # 割り当てられた計算ノードのインスタンス名
```

```bash
gcloud compute ssh --zone $ZONE $INSTANCE --project $PROJECT -- -L 8888:$COMPUTE_INSTANCE:8888
```

ローカルマシンのブラウザで `http://127.0.0.1:8888/lab?token=...` にアクセスして、`!nvidia-smi` などを実行して、結果が表示されれば計算ノード内でJupyterLabが起動していることが確認できる。

確認できたら、JupyterLabをCtrl-cで止めて、計算ノードからログアウトして `squeue` でジョブが終了していることを確認。していなければ `scancel [job_id]` でキャンセルする。

```bash
# Ctrl-cでJupyterLabを止める
exit  # 計算ノードからログアウト
squeue  # ジョブが終了していることを確認。していなければ scancel [job_id] でキャンセル
```

### バッチファイルでJupyterLabを起動

手動でJupyterLabを起動できることが確認出来たので、バッチファイルを作成してジョブとして実行出来るようにする。

松尾研の手順書とマニュアルを参考にする。

- https://docs.google.com/document/d/1kJiRqUFSm6pMFoIX4hqx1CxAIa5MBSwC9DndzB6pxqE/edit?usp=sharing
- https://github.com/matsuolab/ucllm_nedo_prod/blob/main/infra/README.md#batch

`jupyterlab.sh` というファイルを作成して以下の内容を書き込む。**手順書の注意事項を参照してルールの範囲内で設定する**。

```bash
#!/bin/bash

#SBATCH --gpus-per-node=1
#SBATCH --time=06:00:00
#SBATCH --partition=g2
#SBATCH --nodes=1
#SBATCH --job-name=jupyterlab
#SBATCH --output=jupyterlab.log

source $EXP_HOME/miniconda3/etc/profile.d/conda.sh
conda activate jupyter39
# conda activate jupyter310

jupyter-lab --no-browser --port 8888 --ip $(hostname -i)
```

`sbatch` でジョブを投入。

```bash
sbatch jupyterlab.sh
```

`squeue` でジョブを確認。

```bash
squeue
```

`tail -f` でログを確認する。

```bash
tail -f jupyterlab.log
```

ログに `http://127.0.0.1:8888/lab?token=...` のような行が出るので、クリップボードにコピーしておく。

ポートフォワードの設定を前節の手順で行っておく。

ローカルマシンのブラウザで `http://127.0.0.1:8888/lab?token=...` にアクセスして、`!nvidia-smi` などを実行して、結果が表示されれば計算ノード内でJupyterLabが起動していることが確認できる。

JupyterLabでの作業が完了したら必ず `scancel [job_id]` でジョブをキャンセルする。

```bash
# Ctrl-c で tail -f を止める
squeue  # job_id を確認
scancel [job_id]  # ジョブをキャンセル
squeue  # キャンセル出来たか確認
```

## 2回目以降の手順

### sshでログインノードに接続

ローカルマシンのターミナルから以下のコマンドを実行してログインノードに接続。

```bash
export ZONE="asia-southeast1-a"
export INSTANCE="mlpre-login-..."            # ログインノードのインスタンス名
export PROJECT="g..."                        # GCPプロジェクト名
```

```bash
gcloud compute ssh --zone $ZONE $INSTANCE --project $PROJECT
```

### 計算ノードでJupyterLabをバッチ起動

`squeue` して、以前の自分のジョブが終了していることを確認。終了していなければ `scancel [job_id]` でキャンセルする。

JupyterLabをバッチ起動する。

```bash
sbacth jupyterlab.sh
tail -f jupyterlab.log
```

ログに `http://127.0.0.1:8888/lab?token=...` のような行が出るので、クリップボードにコピーしておく。

**今作業しているターミナルとは別のローカルマシンのターミナル**から、以下のコマンドを実行してローカルマシンのポート8888を計算ノード`mlpre-g2-...`のポート8888にフォワードする。`--` 以降のオプションは通常のsshコマンドのオプション。計算ノード名は `squeue` で確認する。

```bash
export ZONE="asia-southeast1-a"
export INSTANCE="mlpre-login-..."       # ログインノードのインスタンス名
export PROJECT="g..."                   # プロジェクト名
export COMPUTE_INSTANCE="mlpre-g2-..."  # 割り当てられた計算ノードのインスタンス名
```

```bash
gcloud compute ssh --zone $ZONE $INSTANCE --project $PROJECT -- -L 8888:$COMPUTE_INSTANCE:8888
```

**実験が終わったら必ず `scancel [job_id]` でジョブをキャンセル**する。

```bash
# Ctrl-c で tail -f を止める
squeue  # job_id を確認
scancel [job_id]  # ジョブをキャンセル
squeue  # キャンセル出来たか確認
```

### ローカルマシンのVSCodeから計算ノードのJupyterLabに接続

ローカルマシンのVSCodeから以下手順で計算ノードのJupyterLabに接続する。計算ノードでのJupyterLabの起動とポートフォワードの設定は事前に行っておく。

VSCodeの手順は以下の通り。

- ノートブックを開いた状態で
- `Command Palette...`
- `Notebook: Select Notebook Kernel`
- `Existing Jupiter Server...`
- コピーしておいた `http://127.0.0.1:8888/lab?token=...` を入力
- `Select Kernel` で `Python 3 (ipykernel)` を選択
- `!nvidia-smi` などを実行してみて結果が表示されれば計算ノードのJupyterLabに接続できているはず

In [ ]:
!nvidia-smi

## Ask-LLM の実行

ここまでの設定は Ask-LLM とは関係なく一般的にJupyterLabを松尾研プレ環境で使うための設定。

ここからは Ask-LLM を実行するための手順。

In [ ]:
# scoring the CulturaX dataset with Ask-LLM

%pip install nano-askllm
%pip install datasets sentencepiece accelerate

In [ ]:
import os
from time import time

from datasets import load_dataset
from nano_askllm import AskLLM
from transformers import AutoModelForCausalLM, AutoTokenizer #, BitsAndBytesConfig

In [ ]:
# you must define the environment variable EXP_HOME or replace it with the path to the directory where the model will be saved
hf_cache_dir = os.path.join(os.environ["EXP_HOME"], ".cache/huggingface/hub")

# I think the `RakutenAI-7B-chat` model tends to say "no" to any kind of question.
# so I use the `RakutenAI-7B-instruct` model instead.
model_id = "Rakuten/RakutenAI-7B-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, torch_dtype="auto", device_map="auto", cache_dir=hf_cache_dir)
# quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
    cache_dir=hf_cache_dir,
    # quantization_config=quantization_config,
)

In [ ]:
dataset = load_dataset(
    "uonlp/CulturaX",
    "ja",
    split="train",
    cache_dir="/persistentshare/storage/team_kumagai/datasets",
)

In [ ]:
prompt_template_prefix = "###\n"
prompt_template_postfix = """
###

Does the previous paragraph demarcated within ### and ### contain informative signal for pre-training a large-language model? An informative datapoint should be well-formatted, contain some usable knowledge of the world, and strictly NOT have any harmful, racist, sexist, etc. content.

OPTIONS: yes/no
ANSWER:"""  # noqa: E501

yes_tokens = ["yes", "Yes"]

llm = AskLLM(
    tokenizer,
    model,
    prompt_template_prefix=prompt_template_prefix,
    prompt_template_postfix=prompt_template_postfix,
    yes_tokens=yes_tokens,
    # you can increase it up to 8192 for Mistral-7B-v0.1 based models
    max_tokens=512,
)

In [ ]:
# you can increase batch_size up to about 16 if max_tokens is 512
# or 8 for 1024, 4 for 2048, 2 for 4096, 1 for 8192, etc.
batch_size = 1
num_ask = 10

start_time = time()

# TODO: rewrite using `map` of `dataset`.
# https://huggingface.co/docs/datasets/process#map
for i in range(num_ask):
    datapoints = dataset[i * batch_size : (i + 1) * batch_size]["text"]
    scores = llm.ask(datapoints)
    for score, datapoint in zip(scores.tolist(), datapoints):
        text = datapoint[:80].replace("\n", "\\n")
        print(f"{score:.4f}\t{text}")
    del scores

end_time = time()
print(f"{(end_time - start_time):.4f} seconds")